# 13 Interfaces, Protocols, and ABCs

## The Typing Map

![The Typing Map](./img/2023-10-04-08-21-17.png)

![Sequence ABCs](./img/2023-10-04-10-14-44.png)

## Two kinds of protocols

- Dynamic protocol: The informal protocols Python always had. Dynamic protocols are implicit, defined by convention, and described in the documentation. Python's most important dynamic protocols are supported by the interpreter itself, and are documented in the ["Data Model" chapter of The Python Language Reference](https://docs.python.org/3/reference/datamodel.html).
- Static protocol: A protocol as defined by [PEP 544--Protocols: Structural subtyping(static duck typing)](https://peps.python.org/pep-0544/), since Python 3.8. A static protocol has explicit definition: a `typing.Protocol` subclass

There are two key differences between them:
- An object may implement only part of dynamic protocol and still be useful; but to fulfill a static protocol, the object must provide every method declared in the protocol class, even if your program doesn't need them all.
- Static protocols can verified by static type checkers, but dynamic protocols can't.


In [3]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

In [4]:
v = Vowels()
v[0]

'A'

In [5]:
v[-1]

'U'

In [6]:
for c in v: print(c)

A
E
I
O
U


In [7]:
len(v)

TypeError: object of type 'Vowels' has no len()

In [18]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])
print(Card._fields)

ranks = [str(n) for n in range(2, 11)] + list('JQKA')
suits = 'spades diamonds clubs hearts'.split()
deck = [Card(rank, suit) for rank in ranks for suit in suits]
print(deck[:5])
print(len(deck))


('rank', 'suit')
[Card(rank='2', suit='spades'), Card(rank='2', suit='diamonds'), Card(rank='2', suit='clubs'), Card(rank='2', suit='hearts'), Card(rank='3', suit='spades')]
52


In [17]:
from random import shuffle
shuffle(deck)
deck[:5]

[Card(rank='A', suit='hearts'),
 Card(rank='5', suit='diamonds'),
 Card(rank='4', suit='clubs'),
 Card(rank='2', suit='diamonds'),
 Card(rank='K', suit='diamonds')]

`str.isidentifier()` method explained

In [21]:
print("hello".isidentifier())      # Output: True
print("123".isidentifier())        # Output: False (starts with a number)
print("_var".isidentifier())       # Output: True
print("if".isidentifier())         # Output: False (reserved keyword)
print("my_function".isidentifier())# Output: True
print("class".isidentifier())      # Output: False (reserved keyword)
print("some-var".isidentifier())   # Output: False (contains '-')


True
False
True
True
True
True
False


`namedtuple` fields must be identifiers.

In [24]:
from collections import namedtuple

# Define a named tuple with valid field names
# '1pass' is not a valid field name
Person = namedtuple('Person', ['name', 'age', 'city', '1pass'])
p = Person('John', 25, 'New York', True)

print(p.name)  # Output: John
print(p.age)   # Output: 25
print(p.city)  # Output: New York

ValueError: Type names and field names must be valid identifiers: '1pass'

## Goose Typing

The Python Glossary entry for [abstract base class](https://docs.python.org/3/glossary.html#term-abstract-base-class) has a good explanation of the value they bring to duck-typed languages:
> Abstract base classes complement duck-typing by providing a way to define interfaces when other techniques like hasattr() would be clumsy or subtly wrong (for example with magic methods). ABCs introduce virtual subclasses, which are classes that don’t inherit from a class but are still recognized by isinstance() and issubclass(); see the abc module documentation. Python comes with many built-in ABCs for data structures (in the collections.abc module), numbers (in the numbers module), streams (in the io module), import finders and loaders (in the importlib.abc module). You can create your own ABCs with the abc module.

To summarize, goose typing entails:
- Subclassing from ABCs to make it explict that you are implementing a previously defined interface.
- Runtime type checking using ABCs instead of concrete classes as the second argument for `isinstance` and `issubclass`
- Don’t define custom ABCs (or metaclasses) in production code

## Subclassing an ABC

In [ ]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self) :
        self._cards = [Card(rank, suit) for rank in self.ranks
                                          for suit in self.suits]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)
        

## ABCs in the Standard Library

The most widely used ABCs are in `collections.abc`

![collections.abc](./img/2023-10-04-10-54-00.png)